## Load necessary modules

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

import os

os.environ['CUDA_VISIBLE_DEVICES'] = str(1)

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
#from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

# use this to change which GPU to use
#gpu = 1

# set the modified tf session as backend in keras
#setup_gpu(gpu)

from keras_retinanet import models

# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('..', 'snapshots', 'resnet152_pascal_02_backup.h5')
dataset_path = "/caa/Homes01/mburges/CVSP-Object-Detection-Historical-Videos/retina_net_video/output/"

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet152')

model = models.convert_model(model)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors


## Load RetinaNet model

In [2]:
# load label to names mapping for visualization purposes
labels_to_names = {0: 'crowd', 1: 'civilian', 2: 'soldier', 3: 'civil vehicle', 4: 'mv'}

## Run detection on example

In [3]:
for filename in os.listdir(dataset_path):
    image = None
    if filename.endswith('.jpg'):

        # Open the file:
        image = cv2.imread(os.path.join(dataset_path,filename))
    if image is not None:
        # copy to draw on
        draw = image.copy()
        draw_regression = image.copy()
        draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

        # preprocess image for network
        image = preprocess_image(image)
        image, scale = resize_image(image)

        # process image
        start = time.time()
        result = model.predict_on_batch(np.expand_dims(image, axis=0))
        boxes, scores, labels = result
        print("processing time: ", time.time() - start)

        # correct for image scale
        boxes /= scale

        # visualize detections
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            # scores are sorted so we can break
            if score < 0.5:
                break

            print (box, label, score)
            color = label_color(label)

            b = box.astype(int)
            draw_box(draw, b, color=color)

            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)

        cv2.imwrite(os.path.join(dataset_path,"detected_"+filename), draw)
#plt.figure(figsize=(17, 17))
#plt.axis('off')
#plt.imshow(draw)
#plt.savefig('/caa/Homes01/mburges/CVSP-Object-Detection-Historical-Videos/result.png')
#plt.show()

processing time:  14.983539581298828
processing time:  0.12204432487487793
processing time:  0.09300637245178223
[607.5637  225.40349 737.20013 603.96277] 1 0.88375086
processing time:  0.09206128120422363
[486.1151  155.2592  717.5609  624.07947] 1 0.52050894
processing time:  0.09435248374938965
processing time:  0.09000372886657715
processing time:  0.0921483039855957
processing time:  0.09550046920776367
[510.8549  137.49898 804.9428  553.4199 ] 2 0.8575064
processing time:  0.09555768966674805
processing time:  0.09353828430175781
processing time:  0.09651637077331543
processing time:  0.09403705596923828
[347.28036  79.86496 527.74695 608.31683] 2 0.5910789
[347.28036  79.86496 527.74695 608.31683] 1 0.517396
processing time:  0.09273624420166016
[425.68863 119.19557 676.1325  627.0878 ] 1 0.5313673
processing time:  0.0914297103881836
[371.56665 168.74414 513.4851  540.01263] 2 0.70219386
[371.80887 168.88281 513.1104  547.34534] 1 0.53447974
processing time:  0.0941653251647949

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



processing time:  0.09222602844238281
Traceback (most recent call last):
  File "/caa/Homes01/mburges/anaconda3/envs/testRetinaNet/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-5e40e6dd715f>", line 14, in <module>
    image = preprocess_image(image)
  File "/caa/Homes01/mburges/anaconda3/envs/testRetinaNet/lib/python3.7/site-packages/keras_retinanet/utils/image.py", line 59, in preprocess_image
    x[..., 1] -= 116.779
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/caa/Homes01/mburges/anaconda3/envs/testRetinaNet/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2034, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

KeyboardInterrupt: 